In [ ]:
import sys, os
sys.path.append(f'../../code')
path_to_json = os.getcwd()

from IPython.display import display, Markdown

In [ ]:
import json
def load_case(path):
    display(Markdown(f'## {path}'))
    with open(path, 'r+') as file: bundle = json.load(file)
    print('Temperature:', bundle['temperature'])
    return bundle

display(Markdown('# Files'))
json_files_mc = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('mc-lT')],
                       reverse = True)
json_files_alg1 = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('a1-lT')],
                         reverse = True)

print("MC: ", json_files_mc)
print("A1: ", json_files_alg1)

mc = load_case(json_files_mc[0])
alg1 = load_case(json_files_alg1[0])

In [ ]:
import ising
from ising import *
import matplotlib.pyplot as plt
import seaborn as sns

fig_kde, axes = plt.subplots(1, 2)
h1 = sns.kdeplot(data=mc["energy_sample"], bw_method='scott', color='m', ax=axes[0])
h1.set(xlabel=None)
h2 = sns.kdeplot(data=alg1["energy_sample"], bw_method='scott', color='b', ax=axes[0])
h2.set(xlabel=None)
m1 = sns.kdeplot(data=mc["magnetization_sample"], bw_method='scott', color='m', ax=axes[1])
m1.set(xlabel=None)
m2 = sns.kdeplot(data=alg1["magnetization_sample"], bw_method='scott', color='b', ax=axes[1])
m2.set(xlabel=None)

axes[0].set_title("Energy")
axes[0].legend(['MonteCarlo', 'Dynamic'], loc="upper left")
axes[1].set_title("Magnetization")
axes[1].legend(['MonteCarlo', 'Dynamic'], loc="upper right")

In [ ]:
chain = Chain(size=100)
bin_edges_engy_mc, metropolis_engy = hist(chain, mc["energy_sample"], engy_flag=True)
bin_edges_engy_dynamic, dynamic_engy = hist(chain, alg1["energy_sample"], engy_flag=True)
fig_engy, axes = plt.subplots(1, 2)
axes[0].hist(metropolis_engy, bins=bin_edges_engy_mc, label='Metropolis', color='m')
axes[1].hist(dynamic_engy, bins=bin_edges_engy_dynamic, label='Dynamic', color='b')
axes[0].set_title("Energy Montecarlo")
axes[1].set_title("Energy Dynamic")
plt.show()

In [ ]:
bin_edges_m_mc, metropolis_m = hist(chain, mc["magnetization_sample"], engy_flag=False)
bin_edges_m_dynamic, dynamic_m = hist(chain, alg1["magnetization_sample"], engy_flag=False)
fig_m, axes = plt.subplots(1, 2)
axes[0].hist(metropolis_m, bins=bin_edges_m_mc, label='Metropolis', color='m')
axes[1].hist(dynamic_m, bins=bin_edges_m_dynamic, label='Dynamic', color='b')
axes[0].set_title("Magnetization Montecarlo")
axes[1].set_title("Magnetization Dynamic")
plt.show()

In [ ]:
mc_engy_corr = acrl(mc["energy_sample"], 100)
mc_m_corr = acrl(mc["magnetization_sample"], 500)
fig_kde, axes = plt.subplots(1, 2)
axes[0].plot(mc_engy_corr)
axes[0].set_title("H Correlation MonteCarlo")
axes[1].plot(mc_m_corr)
axes[1].set_title("M Correlation MonteCarlo")

In [ ]:
alg1_engy_corr = acrl(alg1["energy_sample"], 5000)
alg1_m_corr = acrl(alg1["magnetization_sample"], 8000)
fig_kde, axes = plt.subplots(1, 2)
axes[0].plot(alg1_engy_corr)
axes[0].set_title("H Correlation Alg1")
axes[1].plot(alg1_m_corr)
axes[1].set_title("M Correlation Alg1")

In [ ]:
mc_m_counts = count_variables(mc["magnetization_sample"])
alg1_m_counts = count_variables(alg1["magnetization_sample"])
mc_m_counts, alg1_m_counts = equilibrate_counts(mc_m_counts,alg1_m_counts)
mc_engy_counts = count_variables(mc["energy_sample"])
alg1_engy_counts = count_variables(alg1["energy_sample"])
mc_m_counts, alg1_m_counts = equilibrate_counts(mc_engy_counts,alg1_engy_counts)
#H0: the two sample come from the same distribution
print(f'Energy:', z_test(f1=list(mc_engy_counts.values()), f2=list(alg1_engy_counts.values()), n_samples=len(mc["energy_sample"])))
print(f'Magnetization:', z_test(f1=list(mc_m_counts.values()), f2=list(alg1_m_counts.values()), n_samples=len(mc["energy_sample"])))
